In [1]:
import torch
import json
from pathlib import Path
from task.taskManager import get_model

def load_checkpoint(checkpoint_path):
    checkpoint_path = Path(checkpoint_path)
    assert checkpoint_path.exists()
    checkpoint = torch.load(checkpoint_path)
    task = get_model(checkpoint["arguments"])
    task.net.load_state_dict(checkpoint["model"])
    loss_fn = lambda model, x, y: task.__class__.loss_fn(model, x, y, task.criterion) # we hide the criterion
    forward_fn = task.__class__.forward_fn
    return task.net, forward_fn, loss_fn, task.dataloader_fn, checkpoint["history"]


In [2]:
model, forward_fn, loss_fn, dataloader_fn, history = load_checkpoint("saves/addTask-0-batch-1000.pth")

In [3]:
for batch_num, x, y in dataloader_fn():
    predictions = forward_fn(model, x)
    losses = loss_fn(model, x, y)
    print(predictions)

Variable containing:
 0.8724
 0.8987
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8778
 0.8772
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8858
 0.8770
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8776
 0.8770
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8797
 0.8799
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8800
 0.8793
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8787
 0.8854
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8781
 0.8857
[torch.FloatTensor of size 2x1]

Variable containing:
 0.9010
 0.8835
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8708
 0.9016
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8705
 0.8795
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8707
 0.8798
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8792
 0.8688
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8795
 0.8836
[torch.FloatTensor of size 2x1]

Variable containing:

/Users/julesgagnon-marchand/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Variable containing:
 0.8837
 0.8777
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8794
 0.8793
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8994
 0.8853
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8768
 0.8794
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8797
 0.8702
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8811
 0.8977
[torch.FloatTensor of size 2x1]

Variable containing:
 0.9000
 0.8792
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8997
 0.8837
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8778
 0.8782
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8775
 0.8775
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8711
 0.8861
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8854
 0.8728
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8776
 0.8794
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8776
 0.8975
[torch.FloatTensor of size 2x1]

Variable containing:

Variable containing:
 0.8774
 0.8796
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8910
 0.8855
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8797
 0.8779
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8703
 0.8874
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8862
 0.8691
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8793
 0.8772
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8860
 0.8967
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8709
 0.8774
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8715
 0.8987
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8777
 0.8858
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8783
 0.8817
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8858
 0.8782
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8857
 0.8836
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8992
 0.8708
[torch.FloatTensor of size 2x1]

Variable containing:

Variable containing:
 0.8711
 0.9057
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8853
 0.8974
[torch.FloatTensor of size 2x1]

Variable containing:
 0.9014
 0.8794
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8713
 0.8708
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8812
 0.8775
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8768
 0.8710
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8836
 0.8792
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8998
 0.8758
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8995
 0.8795
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8797
 0.8878
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8815
 0.8817
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8775
 0.8709
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8908
 0.8974
[torch.FloatTensor of size 2x1]

Variable containing:
 0.9059
 0.8854
[torch.FloatTensor of size 2x1]

Variable containing:

Variable containing:
 0.8910
 0.8856
[torch.FloatTensor of size 2x1]

Variable containing:
 0.9057
 0.8794
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8908
 0.8856
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8778
 0.8713
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8775
 0.8705
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8854
 0.8795
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8972
 0.8795
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8799
 0.8691
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8857
 0.8730
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8972
 0.8807
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8811
 0.8795
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8999
 0.8974
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8794
 0.8906
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8997
 0.9014
[torch.FloatTensor of size 2x1]

Variable containing:

Variable containing:
 0.8767
 0.8790
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8878
 0.8798
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8856
 0.8854
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8774
 0.8993
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8694
 0.8799
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8794
 0.8807
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8774
 0.8856
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8706
 0.8711
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8793
 0.8860
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8813
 0.8833
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8792
 0.8690
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8835
 0.8809
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8707
 0.8711
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8714
 0.8724
[torch.FloatTensor of size 2x1]

Variable containing:

Variable containing:
 0.8788
 0.8817
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8997
 0.8975
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8855
 0.8791
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8794
 0.8835
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8704
 0.8861
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8776
 0.8774
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8973
 0.8969
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8771
 0.8991
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8773
 0.8795
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8694
 0.8816
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8772
 0.8970
[torch.FloatTensor of size 2x1]

Variable containing:
 0.9014
 0.8813
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8992
 0.8856
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8770
 0.8854
[torch.FloatTensor of size 2x1]

Variable containing:

Variable containing:
 0.8756
 0.8728
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8781
 0.8794
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8777
 0.8796
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8692
 0.8810
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8775
 0.8995
[torch.FloatTensor of size 2x1]

Variable containing:
 0.9055
 0.8770
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8794
 0.8850
[torch.FloatTensor of size 2x1]

Variable containing:
 0.9054
 0.9004
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8816
 0.8711
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8780
 0.8795
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8690
 0.8693
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8800
 0.8819
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8861
 0.8798
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8797
 0.8689
[torch.FloatTensor of size 2x1]

Variable containing:

Variable containing:
 0.8794
 0.8819
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8996
 0.8772
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8994
 0.8998
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8794
 0.8852
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8839
 0.8856
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8972
 0.8812
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8970
 0.8779
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8792
 0.9012
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8967
 0.8774
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8906
 0.8726
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8770
 0.8802
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8850
 0.9012
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8780
 0.8775
[torch.FloatTensor of size 2x1]

Variable containing:
 0.8796
 0.8781
[torch.FloatTensor of size 2x1]

Variable containing: